In [8]:
#DEBUGGING ONLY: performs the same code as the one below but on a single file.
#WARNING: there are some changes on the version below w.r.t. this one. The current one needs to be aligned with
#         the one below.

import interp_mod
import boolean
import re
import yara

rule_dict = interp_mod.parseString(open("./yara/rules-master/malware/x64dbg/yarasigs-master/crypto_signatures.yara").read())
import_set = set()
include_set = set()
#for each rule found:
for i in range(len(rule_dict)):

    """ sorting the terms of the condition in descending order: this is used in order to
        substitute only the outermost "for" construct in case there are nested ones with a single
        variable (e.g. for i in (0..100):(for j in (0..100):([condition])) --> x0 )"""
    rule_dict[i]["condition_terms"].sort(key= lambda a: -len(a))
    condition = rule_dict[i]["condition"]
    """Conversion to cnf:
            - for each condition term in the sorted list, substitute the condition term with
              a variable name in the condition."""
    for j, el in enumerate(rule_dict[i]["condition_terms"]):
        condition = condition.replace(el,"x"+str(j))
    expr = boolean.BooleanAlgebra()
    bool_expr = expr.parse(condition)
    """ Convert the adjusted expression to cnf and separate the various sub-rules by splitting the
        condition on the and operator."""
    cnf_list = str(expr.cnf(bool_expr)).split("&")
    new_cnf_list = []
    """ each variable is converted back to the original term of the condition."""
    for or_expr in cnf_list:
        literals_list = or_expr.replace("(","").replace(")","").split("|")
        translated_list = []
        for literal in literals_list:
            for cond_number, cond_literal in enumerate(rule_dict[i]["condition_terms"]):
                if literal == "x"+str(cond_number):
                    translated_list.append(cond_literal)
        new_or_expr = " or ".join(translated_list)
        new_cnf_list.append((translated_list,new_or_expr))
    yara_rule_base_string =""
    key_list = rule_dict[i].keys()
    """ Keeping track, for each file, of all the performed imports, so to put them in each sub-rule
        generated (NOTE: there might be more rules for each file, so, imports have to be replied
        for rules belonging to the same file)"""
    if "imports" in key_list:
        for imported_module in rule_dict[i]["imports"]:
            import_set.add(imported_module)
    if len(import_set) > 0:
        for imported_module in import_set:
            yara_rule_base_string+= "import "+imported_module+"\n"
        yara_rule_base_string+="\n"
    if "includes" in key_list:
        for include in rule_dict[i]["includes"]:
            include_set.add(include)
    if len(include_set) > 0:
        yara_rule_base_string+="include "+ include +"\n"
        yara_rule_base_string+="\n\n"
    """ Keeping track of the scopes of the rules. 
        NOTE: these rules could easily be skipped ( it might not be the best idea to split those rules,
              being only a support for others)"""
    if "scopes" in key_list:
        """for scope in rule_dict[i]["scopes"]:
            yara_rule_base_string+=scope+" "
        """
        continue
    yara_rule_base_string+="rule "+rule_dict[i]["rule_name"]+"%s{\n"
    """ For each found "sub-rule" (chunk in between two "and") of a given rule, create a new yara rule."""
    for j, entry in enumerate(new_cnf_list):
        yara_rule = yara_rule_base_string % ("_"+str(j))
        """ If the rule does have some strings declaration, find the required ones
            and include them in the new rule."""
        if "strings" in key_list:
            required_strings = ""
            """ Creating a set just to avoid importing the same string more than once."""
            string_set = set()
            for literal in entry[0]:
                """ Removing array indexing ([]), strings("") and regex (//), so not to match
                    strings inside them (array indexing might still be useful?).
                    Replacing each special character referring to strings with '$' ('#','!','@')."""
                clean_string = re.sub(r'"[^"]*"|/[^/]*/|\[[^\]]\]'," ", literal).replace("#","$").replace("!","$").replace("@","$")
                """ If the modified string still contains "of them" token, it means that all the strings
                    are required, so, add all of them to the set and skip the next for iteration."""
                if(" of them" in clean_string):
                    for string in rule_dict[i]["strings"]:
                        string_to_include=string["name"]+" = "+string["value"]+" "
                        if("modifiers" in string.keys()):
                            for modifier in string["modifiers"]:
                                string_to_include+=modifier+" "
                        string_set.add(string_to_include)  
                    break
                """ Else, compile regex to match string IDs and find all of them in the clean string."""
                clean_literal_list = re.compile("\$[a-zA-Z0-9_\*]*").findall(clean_string)
                for clean_literal in clean_literal_list:
                    """ Creating the proper string to apply a regex (substitute '*' with '.*' and
                        '$' with '\$') and adding a final whitespace so to avoid matching longer
                        strings starting as the current literal (e.g. $dat and $dataset)."""
                    clean_literal = clean_literal.replace("$","\\$").replace("*",".*")+" "
                    for string in rule_dict[i]["strings"]:
                        if re.match(clean_literal,string["name"]+" "):
                            string_to_include=string["name"]+" = "+string["value"]+" "
                            if("modifiers" in string.keys()):
                                for modifier in string["modifiers"]:
                                    string_to_include+=modifier+" "
                            """ Add the string and its modifiers to the set of all required strings."""
                            string_set.add(string_to_include)   
                            break
            """ Composing the 'strings:' section of the yara rule."""
            for string in string_set:
                required_strings+="\t\t"+string+"\n"
            if required_strings != "":
                yara_rule+="\tstrings:\n"
                yara_rule+=required_strings
                yara_rule+="\n"
        yara_rule+="\tcondition:\n"
        """ Adding the condition of the sub-rule"""
        yara_rule += "\t\t"+entry[1]+"\n}" 
        print(yara_rule)
        """ DEBUGGING ONLY."""
        yara.compile(source=yara_rule)

rule Prime_Constants_char_0{
	strings:
		$c0 = { 03 05 07 0B 0D 11 13 17 1D 1F 25 29 2B 2F 35 3B 3D 43 47 49 4F 53 59 61 65 67 6B 6D 71 7F 83 89 8B 95 97 9D A3 A7 AD B3 B5 BF C1 C5 C7 D3 DF E3 E5 E9 EF F1 FB } 

	condition:
		$c0
}
rule Prime_Constants_long_0{
	strings:
		$c0 = { 03 00 00 00 05 00 00 00 07 00 00 00 0B 00 00 00 0D 00 00 00 11 00 00 00 13 00 00 00 17 00 00 00 1D 00 00 00 1F 00 00 00 25 00 00 00 29 00 00 00 2B 00 00 00 2F 00 00 00 35 00 00 00 3B 00 00 00 3D 00 00 00 43 00 00 00 47 00 00 00 49 00 00 00 4F 00 00 00 53 00 00 00 59 00 00 00 61 00 00 00 65 00 00 00 67 00 00 00 6B 00 00 00 6D 00 00 00 71 00 00 00 7F 00 00 00 83 00 00 00 89 00 00 00 8B 00 00 00 95 00 00 00 97 00 00 00 9D 00 00 00 A3 00 00 00 A7 00 00 00 AD 00 00 00 B3 00 00 00 B5 00 00 00 BF 00 00 00 C1 00 00 00 C5 00 00 00 C7 00 00 00 D3 00 00 00 DF 00 00 00 E3 00 00 00 E5 00 00 00 E9 00 00 00 EF 00 00 00 F1 00 00 00 FB 00 00 00 } 

	condition:
		$c0
}
rule Advapi_Hash_API_0{
	strings:
		$CryptAcquireContext = 

In [9]:
import interp_mod
import boolean
import re
import yara
import os


path = "./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/"
for root, dirs, files in os.walk(path):
    for filename in files:
        try:
            #print(os.path.join(root,filename))
            
            #parsing the rule file into a dictionary.
            rule_dict = interp_mod.parseString(open(os.path.join(root, filename)).read())
            import_set = set()
            include_set = set()
            #for each rule found:
            for i in range(len(rule_dict)):
                
                """ sorting the terms of the condition in descending order: this is used in order to
                    substitute only the outermost "for" construct in case there are nested ones with a single
                    variable (e.g. for i in (0..100):(for j in (0..100):([condition])) --> x0 )"""
                rule_dict[i]["condition_terms"].sort(key= lambda a: -len(a))
                condition = rule_dict[i]["condition"]
                """Conversion to cnf:
                        - for each condition term in the sorted list, substitute the condition term with
                          a variable name in the condition."""
                for j, el in enumerate(rule_dict[i]["condition_terms"]):
                    condition = condition.replace(el,"x"+str(j))
                expr = boolean.BooleanAlgebra()
                bool_expr = expr.parse(condition)
                """ Convert the adjusted expression to cnf and separate the various sub-rules by splitting the
                    condition on the and operator."""
                cnf_list = str(expr.cnf(bool_expr)).split("&")
                new_cnf_list = []
                """ each variable is converted back to the original term of the condition."""
                for or_expr in cnf_list:
                    literals_list = or_expr.replace("(","").replace(")","").split("|")
                    translated_list = []
                    for literal in literals_list:
                        for cond_number, cond_literal in enumerate(rule_dict[i]["condition_terms"]):
                            if literal == "x"+str(cond_number):
                                translated_list.append(cond_literal)
                    new_or_expr = " or ".join(translated_list)
                    new_cnf_list.append((translated_list,new_or_expr))
                yara_rule_base_string =""
                key_list = rule_dict[i].keys()
                """ Keeping track, for each file, of all the performed imports, so to put them in each sub-rule
                    generated (NOTE: there might be more rules for each file, so, imports have to be replied
                    for rules belonging to the same file)"""
                if "imports" in key_list:
                    for imported_module in rule_dict[i]["imports"]:
                        import_set.add(imported_module)
                if len(import_set) > 0:
                    for imported_module in import_set:
                        yara_rule_base_string+= "import "+imported_module+"\n"
                    yara_rule_base_string+="\n"
                if "includes" in key_list:
                    for include in rule_dict[i]["includes"]:
                        include_set.add(include)
                if len(include_set) > 0:
                    yara_rule_base_string+="include "+ include +"\n"
                    yara_rule_base_string+="\n\n"
                """ Keeping track of the scopes of the rules. 
                    NOTE: these rules could easily be skipped ( it might not be the best idea to split those rules,
                          being only a support for others)"""
                if "scopes" in key_list:
                    """for scope in rule_dict[i]["scopes"]:
                        yara_rule_base_string+=scope+" "
                    """
                    continue
                yara_rule_base_string+="rule "+rule_dict[i]["rule_name"]+"%s{\n"
                """ For each found "sub-rule" (chunk in between two "and") of a given rule, create a new yara rule."""
                for j, entry in enumerate(new_cnf_list):
                    yara_rule = yara_rule_base_string % ("_"+str(j))
                    """ If the rule does have some strings declaration, find the required ones
                        and include them in the new rule."""
                    if "strings" in key_list:
                        required_strings = ""
                        """ Creating a set just to avoid importing the same string more than once."""
                        string_set = set()
                        for literal in entry[0]:
                            """ Removing array indexing ([]), strings("") and regex (//), so not to match
                                strings inside them (array indexing might still be useful?).
                                Replacing each special character referring to strings with '$' ('#','!','@')."""
                            clean_string = re.sub(r'"[^"]*"|/[^/]*/|\[[^\]]\]'," ", literal).replace("#","$").replace("!","$").replace("@","$")
                            """ If the modified string still contains "of them" token, it means that all the strings
                                are required, so, add all of them to the set and skip the next for iteration."""
                            if(" of them" in clean_string):
                                for string in rule_dict[i]["strings"]:
                                    string_to_include=string["name"]+" = "+string["value"]+" "
                                    if("modifiers" in string.keys()):
                                        for modifier in string["modifiers"]:
                                            string_to_include+=modifier+" "
                                    string_set.add(string_to_include)  
                                break
                            """ Else, compile regex to match string IDs and find all of them in the clean string."""
                            clean_literal_list = re.compile("\$[a-zA-Z0-9_\*]*").findall(clean_string)
                            for clean_literal in clean_literal_list:
                                """ Creating the proper string to apply a regex (substitute '*' with '.*' and
                                    '$' with '\$') and adding a final whitespace so to avoid matching longer
                                    strings starting as the current literal (e.g. $dat and $dataset)."""
                                clean_literal = clean_literal.replace("$","\\$").replace("*",".*")+" "
                                for string in rule_dict[i]["strings"]:
                                    if re.match(clean_literal,string["name"]+" "):
                                        string_to_include=string["name"]+" = "+string["value"]+" "
                                        if("modifiers" in string.keys()):
                                            for modifier in string["modifiers"]:
                                                string_to_include+=modifier+" "
                                        """ Add the string and its modifiers to the set of all required strings."""
                                        string_set.add(string_to_include)   
                                        break
                        """ Composing the 'strings:' section of the yara rule."""
                        for string in string_set:
                            required_strings+="\t\t"+string+"\n"
                        if required_strings != "":
                            yara_rule+="\tstrings:\n"
                            yara_rule+=required_strings
                            yara_rule+="\n"
                    yara_rule+="\tcondition:\n"
                    """ Adding the condition of the sub-rule"""
                    yara_rule += "\t\t"+entry[1]+"\n}" 
                    print(yara_rule)
                    """ DEBUGGING ONLY."""
                    yara.compile(source=yara_rule)

        except Exception as e:
            print(root,filename)
            print(e)

./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/malware-signatures-4baa68edd15ebc72d6d049331cd1140973ad1c76 README.md
unknown text at 'Malware' ; token of type 'ID'
./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/malware-signatures-4baa68edd15ebc72d6d049331cd1140973ad1c76/packrat md5-c2.csv
unknown text at 'MD5' ; token of type 'ID'
./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/malware-signatures-4baa68edd15ebc72d6d049331cd1140973ad1c76/packrat domains.csv
unknown text at 'Domain' ; token of type 'ID'
./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/malware-signatures-4baa68edd15ebc72d6d049331cd1140973ad1c76/packrat README.md
unknown text at 'These' ; token of type 'ID'
./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/malware-signatures-4baa68edd15ebc72d6d049331cd1140973ad1c76/packrat assorted.csv
unknown text at 'android' ; token of type 'ID'
./yara/rules-master/malware/x64dbg/yarasigs-master/citizenlab/malware-sign